<a href="https://colab.research.google.com/github/financieras/big_data/blob/main/leccion_2_1_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lección 2.1.3: Detección de patrones y outliers

## 1. El arte de encontrar lo extraordinario en lo ordinario

Detectar patrones y outliers es como ser un **arqueólogo de datos**: excavas capas de información para descubrir tesoros escondidos y anomalías que cuentan historias importantes. Los patrones revelan "cómo funciona el mundo normal", los outliers te muestran "donde el mundo se rompe o innova".

> **Idea clave:** Un outlier no es un error—es una oportunidad disfrazada de anomalía.

**¿Por qué dedicar tiempo a esto?**
- 🔍 **Patrones** → Predicción, automatización, comprensión del negocio
- 🚨 **Outliers** → Fraudes, errores, oportunidades, innovaciones

**Ejemplo dramático:** En 2012, un analista en Netflix detectó un patrón inusual: usuarios que veían series completas en 48 horas. Eran outliers estadísticos, pero representaban un **nuevo comportamiento de consumo** que llevó al desarrollo del binge-watching y cambió la industria del streaming.

> **Advertencia crítica:** Un outlier puede ser un error que arruina tu análisis O una oportunidad que vale millones. La clave es **distinguirlos**.

---

## 2. Detección de patrones: Encontrando el ritmo en el ruido

### **Tipos de patrones principales**

| Tipo | Descripción | Ejemplo | Visualización |
|------|-------------|---------|---------------|
| **Tendencia** | Dirección general a largo plazo | Aumento de ventas año tras año | Line plot ascendente |
| **Estacionalidad** | Repetición periódica regular | Ventas altas cada diciembre | Line plot con ciclos |
| **Clustering** | Agrupaciones naturales | Clientes de alto/bajo valor | Scatter plot con grupos |
| **Correlación** | Relación entre variables | Temperatura vs ventas helado | Scatter plot lineal |
| **Secuencia** | Eventos que ocurren juntos | Productos comprados juntos | Network graph |

### **Patrones temporales: El latido del negocio**

```python
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

def patrones_temporales(df, fecha_col, valor_col):
    """Detectar patrones diarios, semanales y horarios"""
    df_temp = df.copy()
    df_temp['dia_semana'] = df_temp[fecha_col].dt.day_name()
    df_temp['hora'] = df_temp[fecha_col].dt.hour
    df_temp['mes'] = df_temp[fecha_col].dt.month
    
    # Patrón por día de semana
    patron_diario = df_temp.groupby('dia_semana')[valor_col].mean()
    
    # Patrón por hora
    patron_horario = df_temp.groupby('hora')[valor_col].mean()
    
    # Patrón mensual (estacionalidad)
    patron_mensual = df_temp.groupby('mes')[valor_col].mean()
    
    # Visualización
    fig, axes = plt.subplots(1, 3, figsize=(15, 4))
    
    patron_diario.plot(kind='bar', ax=axes[0], color='skyblue')
    axes[0].set_title('Patrón Semanal')
    axes[0].set_xlabel('Día de la semana')
    
    patron_horario.plot(kind='line', ax=axes[1], marker='o', color='orange')
    axes[1].set_title('Patrón Horario')
    axes[1].set_xlabel('Hora del día')
    
    patron_mensual.plot(kind='bar', ax=axes[2], color='green')
    axes[2].set_title('Patrón Estacional (Mensual)')
    axes[2].set_xlabel('Mes')
    
    plt.tight_layout()
    plt.show()
    
    return patron_diario, patron_horario, patron_mensual

# Uso en ventas de e-commerce
ventas_dia, ventas_hora, ventas_mes = patrones_temporales(
    df, 'fecha', 'monto'
)

print("\n=== PATRONES DETECTADOS ===")
print("📈 Lunes: Peak de ventas post-fin de semana")
print("📉 Miércoles: Valle semanal típico")
print("🎯 Viernes tarde: Compras para el fin de semana")
print("🌙 Noches: Mobile shopping dominance")
```

### **Patrones de agrupación natural (Clustering)**

```python
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

def detectar_grupos_naturales(df, columnas):
    """Encontrar agrupaciones naturales en los datos"""
    # Estandarizar datos
    scaler = StandardScaler()
    datos_escalados = scaler.fit_transform(df[columnas])
    
    # Método del codo para encontrar k óptimo
    inercias = []
    K_range = range(2, 9)
    for k in K_range:
        kmeans = KMeans(n_clusters=k, random_state=42)
        kmeans.fit(datos_escalados)
        inercias.append(kmeans.inertia_)
    
    # Visualizar método del codo
    plt.figure(figsize=(8, 5))
    plt.plot(K_range, inercias, marker='o', linestyle='-', color='b')
    plt.xlabel('Número de clusters (k)')
    plt.ylabel('Inercia')
    plt.title('Método del Codo para Elegir K')
    plt.grid(True)
    plt.show()
    
    # Aplicar K-means con k óptimo (según gráfico)
    k_optimo = 3  # Ajustar según el codo visual
    kmeans = KMeans(n_clusters=k_optimo, random_state=42)
    df['cluster'] = kmeans.fit_predict(datos_escalados)
    
    # Análisis de clusters
    print("\n=== ANÁLISIS DE CLUSTERS ===")
    for i in range(k_optimo):
        cluster_data = df[df['cluster'] == i]
        print(f"\nCluster {i} (n={len(cluster_data)}):")
        print(cluster_data[columnas].describe())
    
    return df, kmeans

# Uso: segmentación de clientes
df_segmentado, modelo = detectar_grupos_naturales(
    df,
    ['ingreso_total', 'frecuencia_compra', 'antiguedad_dias']
)
```

### **Patrones de correlación**

```python
def analizar_correlaciones(df, umbral=0.7):
    """Encuentra correlaciones fuertes entre variables"""
    # Solo variables numéricas
    df_numerico = df.select_dtypes(include=[np.number])
    
    # Matriz de correlación
    correlacion = df_numerico.corr()
    
    # Visualización
    plt.figure(figsize=(10, 8))
    sns.heatmap(correlacion, annot=True, cmap='coolwarm',
                center=0, fmt='.2f', square=True)
    plt.title('Matriz de Correlación')
    plt.show()
    
    # Encontrar correlaciones fuertes
    corr_abs = correlacion.abs()
    upper = corr_abs.where(
        np.triu(np.ones(corr_abs.shape), k=1).astype(bool)
    )
    
    fuertes = []
    for column in upper.columns:
        for row in upper.index:
            if upper.loc[row, column] > umbral:
                fuertes.append((row, column, correlacion.loc[row, column]))
    
    print("\n=== CORRELACIONES FUERTES ===")
    for var1, var2, valor in fuertes:
        print(f"{var1} <-> {var2}: {valor:.3f}")
    
    return correlacion, fuertes

# Uso
correlaciones, pares_fuertes = analizar_correlaciones(df, umbral=0.7)
```

---

## 3. Detección de outliers: Cazando anomalías

### **Clasificación de outliers**

| Tipo | Definición | Causa común | Acción típica |
|------|------------|-------------|---------------|
| **Error de medición** | Dato mal capturado | Error humano, sensor roto | Eliminar o corregir |
| **Error de procesamiento** | Dato mal transformado | Bug en código, formato incorrecto | Corregir pipeline |
| **Evento genuino raro** | Dato real pero extremo | Cliente VIP, Black Friday | **Mantener y analizar** |
| **Fraude/Anomalía** | Dato sospechoso | Fraude, ataque, manipulación | Investigar y separar |

### **Método 1: Rango Intercuartílico (IQR) - El clásico**

```python
def detectar_outliers_iqr(df, columna):
    """Detecta outliers usando el método IQR"""
    Q1 = df[columna].quantile(0.25)
    Q3 = df[columna].quantile(0.75)
    IQR = Q3 - Q1
    
    limite_inferior = Q1 - 1.5 * IQR
    limite_superior = Q3 + 1.5 * IQR
    
    outliers = df[(df[columna] < limite_inferior) |
                  (df[columna] > limite_superior)]
    
    print(f"\n=== DETECCIÓN IQR: {columna} ===")
    print(f"Q1: {Q1:.2f}, Q3: {Q3:.2f}, IQR: {IQR:.2f}")
    print(f"Límites: [{limite_inferior:.2f}, {limite_superior:.2f}]")
    print(f"Outliers: {len(outliers)} ({len(outliers)/len(df)*100:.1f}%)")
    
    return outliers, limite_inferior, limite_superior

# Uso y visualización
outliers_precio, lim_inf, lim_sup = detectar_outliers_iqr(df, 'precio')

plt.figure(figsize=(12, 5))

# Boxplot
plt.subplot(1, 2, 1)
plt.boxplot(df['precio'], vert=False)
plt.xlabel('Precio')
plt.title('Boxplot: Detección de Outliers')

# Histograma con límites
plt.subplot(1, 2, 2)
plt.hist(df['precio'], bins=50, alpha=0.7)
plt.axvline(lim_inf, color='red', linestyle='--', label='Límite inferior')
plt.axvline(lim_sup, color='red', linestyle='--', label='Límite superior')
plt.xlabel('Precio')
plt.ylabel('Frecuencia')
plt.title('Histograma con Límites IQR')
plt.legend()

plt.tight_layout()
plt.show()
```

### **Método 2: Z-Score - Para distribuciones normales**

```python
from scipy import stats

def detectar_outliers_zscore(df, columna, umbral=3):
    """Detecta outliers usando Z-score"""
    z_scores = np.abs(stats.zscore(df[columna].dropna()))
    outliers_mask = z_scores > umbral
    outliers = df.loc[df[columna].notna()].iloc[np.where(outliers_mask)[0]]
    
    print(f"\n=== DETECCIÓN Z-SCORE: {columna} ===")
    print(f"Umbral: {umbral} desviaciones estándar")
    print(f"Outliers: {len(outliers)} ({len(outliers)/len(df)*100:.1f}%)")
    
    return outliers

# Interpretación del Z-score:
# |Z| < 2 → Normal (95% de los datos)
# 2 < |Z| < 3 → Atípico
# |Z| > 3 → Outlier extremo

outliers_zscore = detectar_outliers_zscore(df, 'precio', umbral=3)
```

### **Método 3: Isolation Forest - Para datos multivariados**

```python
from sklearn.ensemble import IsolationForest

def detectar_outliers_multivariados(df, columnas, contaminacion=0.05):
    """Detecta outliers considerando múltiples variables"""
    X = df[columnas].dropna()
    
    # Entrenar modelo
    iso_forest = IsolationForest(
        contamination=contaminacion,
        random_state=42,
        n_estimators=100
    )
    predicciones = iso_forest.fit_predict(X)
    
    # -1 indica outlier, 1 indica normal
    df_resultado = df.loc[X.index].copy()
    df_resultado['es_outlier'] = predicciones == -1
    outliers = df_resultado[df_resultado['es_outlier']]
    
    print(f"\n=== ISOLATION FOREST ===")
    print(f"Variables analizadas: {columnas}")
    print(f"Outliers multivariados: {len(outliers)} ({contaminacion*100}% esperado)")
    
    return outliers, iso_forest

# Uso: detectar outliers considerando precio, cantidad y descuento
outliers_multi, modelo = detectar_outliers_multivariados(
    df,
    columnas=['precio', 'cantidad', 'descuento'],
    contaminacion=0.03
)
```

### **Método 4: DBSCAN - Clustering con outliers**

```python
from sklearn.cluster import DBSCAN

def clustering_con_outliers(df, columnas, eps=0.5, min_samples=5):
    """DBSCAN detecta clusters y outliers automáticamente"""
    # Estandarizar
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(df[columnas])
    
    # Aplicar DBSCAN
    dbscan = DBSCAN(eps=eps, min_samples=min_samples)
    clusters = dbscan.fit_predict(X_scaled)
    
    # -1 indica outliers (noise)
    df_resultado = df.copy()
    df_resultado['cluster'] = clusters
    df_resultado['es_outlier'] = clusters == -1
    
    n_outliers = sum(clusters == -1)
    n_clusters = len(set(clusters)) - (1 if -1 in clusters else 0)
    
    print(f"\n=== DBSCAN ===")
    print(f"Clusters encontrados: {n_clusters}")
    print(f"Outliers (noise): {n_outliers} ({n_outliers/len(df)*100:.1f}%)")
    
    return df_resultado, dbscan

# Uso
df_dbscan, modelo_dbscan = clustering_con_outliers(
    df, ['precio', 'cantidad'], eps=0.5, min_samples=5
)
```

---

## 4. Caso práctico: Transacciones financieras

**Contexto:** Dataset de 50,000 transacciones de tarjetas de crédito. Buscamos patrones de gasto y transacciones fraudulentas.

```python
import pandas as pd
import numpy as np

# 1. CARGAR Y PREPARAR DATOS
transacciones = pd.read_csv('transacciones.csv',
                            parse_dates=['fecha_transaccion'])
transacciones['hora'] = transacciones['fecha_transaccion'].dt.hour
transacciones['dia_semana'] = transacciones['fecha_transaccion'].dt.day_name()

print(f"Total transacciones: {len(transacciones)}")

# 2. ANÁLISIS DE PATRONES POR CATEGORÍA
gasto_por_categoria = transacciones.groupby('categoria')['monto'].agg([
    'mean', 'std', 'count', 'min', 'max'
])
print("\n=== PATRÓN DE GASTO POR CATEGORÍA ===")
print(gasto_por_categoria.sort_values('mean', ascending=False))

# 3. DETECCIÓN DE OUTLIERS POR CATEGORÍA
def outliers_por_categoria(df, categoria_col, monto_col):
    """Detectar outliers dentro de cada categoría"""
    resultados = {}
    
    for categoria in df[categoria_col].unique():
        subset = df[df[categoria_col] == categoria]
        outliers_cat, lim_inf, lim_sup = detectar_outliers_iqr(
            subset, monto_col
        )
        
        resultados[categoria] = {
            'outliers': outliers_cat,
            'n_transacciones': len(subset),
            'n_outliers': len(outliers_cat),
            'porcentaje': len(outliers_cat) / len(subset) * 100,
            'limite_superior': lim_sup
        }
    
    return resultados

outliers_por_cat = outliers_por_categoria(
    transacciones, 'categoria', 'monto'
)

# 4. IDENTIFICAR CATEGORÍAS SOSPECHOSAS
print("\n=== ANÁLISIS DE RIESGO POR CATEGORÍA ===")
for categoria, info in outliers_por_cat.items():
    if info['porcentaje'] > 5:
        print(f"⚠️  ALERTA: {categoria}")
        print(f"   - {info['porcentaje']:.1f}% de outliers")
        print(f"   - {info['n_outliers']} transacciones sospechosas")

# 5. MARCAR OUTLIERS EN DATASET COMPLETO
transacciones['es_outlier'] = False
for categoria, info in outliers_por_cat.items():
    indices_outliers = info['outliers'].index
    transacciones.loc[indices_outliers, 'es_outlier'] = True

# 6. COMPARAR PATRONES: NORMALES VS OUTLIERS
patron_normal = transacciones[~transacciones['es_outlier']].groupby('hora').size()
patron_outliers = transacciones[transacciones['es_outlier']].groupby('hora').size()

plt.figure(figsize=(12, 6))
plt.plot(patron_normal.index, patron_normal.values,
         label='Transacciones Normales', marker='o', linewidth=2)
plt.plot(patron_outliers.index, patron_outliers.values,
         label='Outliers (Sospechosas)', marker='s', linewidth=2, color='red')
plt.xlabel('Hora del día')
plt.ylabel('Número de transacciones')
plt.title('Patrón Horario: Normales vs Outliers')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

# 7. ESTADÍSTICAS FINALES
print("\n=== RESULTADO FINAL ===")
print(f"Transacciones analizadas: {len(transacciones)}")
print(f"Outliers detectados: {transacciones['es_outlier'].sum()}")
print(f"Porcentaje de outliers: {transacciones['es_outlier'].mean()*100:.1f}%")
```

**Hallazgos del caso:**
- 🎯 **Restaurantes de lujo:** 12% de outliers (eventos especiales + posibles fraudes)
- ⏰ **Concentración horaria:** Outliers entre 2-5 AM (horario inusual para transacciones grandes)
- 💡 **Patrón descubierto:** Los outliers en "Electrónicos" seguían patrón de compras corporativas, no fraude
- 🚨 **Acción:** 25 transacciones reportadas al equipo antifraude

---

## 5. Matriz de decisión: ¿Qué hago con los outliers?

### **Framework de análisis**

| Pregunta | Respuesta | Acción |
|----------|-----------|--------|
| ¿Es físicamente posible? | NO | **ELIMINAR** - Error de medición |
| ¿Tiene sentido en el negocio? | NO | **INVESTIGAR** - Posible fraude |
| ¿Es un evento genuino raro? | SÍ | **MANTENER** - Información valiosa |
| ¿Afecta significativamente al análisis? | SÍ | **SEPARAR** - Segmento especial |

### **Estrategias de tratamiento**

```python
# ELIMINAR outliers (errores confirmados)
df_limpio = df[~df['es_outlier']]

# WINSORIZAR (cap a percentiles - suavizar extremos)
percentil_1 = df['precio'].quantile(0.01)
percentil_99 = df['precio'].quantile(0.99)
df['precio_winsorizado'] = df['precio'].clip(percentil_1, percentil_99)

# TRANSFORMAR (log para reducir impacto)
df['precio_log'] = np.log1p(df['precio'])  # log(1+x) evita log(0)

# SEPARAR en segmentos
df['segmento'] = np.where(
    df['precio'] > df['precio'].quantile(0.99),
    'VIP',
    'Regular'
)

# MANTENER con variable dummy
df['es_outlier_precio'] = df['precio'] > limite_superior
```

---

## 6. Técnicas avanzadas: Patrones especiales

### **Descomposición estacional**

```python
from statsmodels.tsa.seasonal import seasonal_decompose

def analizar_estacionalidad(serie_temporal, periodo=30):
    """Descomponer serie en tendencia, estacionalidad y residuales"""
    resultado = seasonal_decompose(
        serie_temporal,
        model='additive',  # 'multiplicative' para crecimientos %
        period=periodo
    )
    
    # Visualización
    fig, axes = plt.subplots(4, 1, figsize=(12, 10))
    
    resultado.observed.plot(ax=axes[0], title='Serie Original')
    axes[0].set_ylabel('Observado')
    
    resultado.trend.plot(ax=axes[1], title='Tendencia')
    axes[1].set_ylabel('Tendencia')
    
    resultado.seasonal.plot(ax=axes[2], title='Componente Estacional')
    axes[2].set_ylabel('Estacionalidad')
    
    resultado.resid.plot(ax=axes[3], title='Residuales (Outliers potenciales)')
    axes[3].set_ylabel('Residuales')
    axes[3].axhline(y=0, color='r', linestyle='--', alpha=0.5)
    
    plt.tight_layout()
    plt.show()
    
    # Outliers en residuales
    residuales = resultado.resid.dropna()
    outliers_residuales = detectar_outliers_zscore(
        pd.DataFrame({'residual': residuales}),
        'residual',
        umbral=2
    )
    
    print(f"Outliers temporales detectados: {len(outliers_residuales)}")
    
    return resultado

# Uso con ventas diarias
ventas_diarias = df.groupby('fecha')['ventas'].sum()
resultado_estacional = analizar_estacionalidad(ventas_diarias, periodo=7)
```

### **Market Basket Analysis (Patrones de secuencia)**

```python
def patrones_compras_conjuntas(transacciones):
    """Encontrar productos que se compran juntos"""
    from mlxtend.frequent_patterns import apriori, association_rules
    
    # Crear matriz one-hot (basket format)
    basket = transacciones.groupby(['id_transaccion', 'producto'])['cantidad'].sum()
    basket = basket.unstack().fillna(0)
    basket = basket.applymap(lambda x: 1 if x > 0 else 0)
    
    # Itemsets frecuentes
    itemsets_frecuentes = apriori(basket, min_support=0.01, use_colnames=True)
    
    # Reglas de asociación
    reglas = association_rules(
        itemsets_frecuentes,
        metric="lift",
        min_threshold=1.0
    )
    
    # Top reglas
    top_reglas = reglas.sort_values('lift', ascending=False).head(10)
    
    print("\n=== TOP 10 PATRONES DE COMPRA ===")
    for idx, row in top_reglas.iterrows():
        print(f"{list(row['antecedents'])} → {list(row['consequents'])}")
        print(f"  Lift: {row['lift']:.2f}, Confianza: {row['confidence']:.2%}\n")
    
    return reglas

# Uso
reglas_asociacion = patrones_compras_conjuntas(transacciones)
```

---

## 7. Pipeline completo de análisis

```python
def pipeline_patrones_outliers(df):
    """Pipeline completo para análisis de patrones y outliers"""
    resultados = {}
    
    print("=== INICIANDO ANÁLISIS COMPLETO ===\n")
    
    # 1. Patrones temporales
    if 'fecha' in df.columns:
        print("1. Analizando patrones temporales...")
        resultados['patrones_temporales'] = patrones_temporales(
            df, 'fecha', 'valor'
        )
    
    # 2. Outliers univariados
    print("\n2. Detectando outliers univariados...")
    columnas_numericas = df.select_dtypes(include=[np.number]).columns
    resultados['outliers_univariados'] = {}
    
    for col in columnas_numericas[:3]:  # Primeras 3 para no sobrecargar
        outliers, _, _ = detectar_outliers_iqr(df, col)
        resultados['outliers_univariados'][col] = outliers
    
    # 3. Outliers multivariados
    if len(columnas_numericas) >= 2:
        print("\n3. Detectando outliers multivariados...")
        resultados['outliers_multivariados'], _ = detectar_outliers_multivariados(
            df, columnas_numericas.tolist()[:3]
        )
    
    # 4. Clustering
    print("\n4. Analizando clustering...")
    resultados['clustering'], _ = detectar_grupos_naturales(
        df, columnas_numericas.tolist()[:3]
    )
    
    # 5. Correlaciones
    print("\n5. Analizando correlaciones...")
    resultados['correlaciones'], resultados['correlaciones_fuertes'] = analizar_correlaciones(df)
    
    print("\n=== ANÁLISIS COMPLETO ===")
    return resultados

# Uso
resultados_completos = pipeline_patrones_outliers(df)
```

---

## 8. Buenas prácticas

### ✅ **Hacer siempre:**

```python
# Documentar umbrales y decisiones
# IQR con factor 1.5 para moderados, 3.0 para extremos
outliers = detectar_outliers_iqr(df, 'precio')  # Factor 1.5 por defecto

# Considerar contexto de negocio
# Un outlier en diciembre puede ser normal (Navidad)

# Visualizar antes de decidir
visualizar_outliers(df, 'variable_critica')

# Comparar antes/después
print(f"Media antes: {df['precio'].mean():.2f}")
df_limpio = df[~df['es_outlier']]
print(f"Media después: {df_limpio['precio'].mean():.2f}")
```

### ❌ **Evitar:**

```python
# ❌ NO eliminar automáticamente
# df = df[z_scores < 3]  # PELIGRO: Pérdida de información valiosa

# ❌ NO usar Z-score en distribuciones no normales
# Verifica normalidad primero

# ❌ NO ignorar el clustering natural
# Los outliers pueden formar su propio cluster válido
```

### **Checklist de análisis**

- [ ] Patrones temporales identificados (día, hora, mes)
- [ ] Outliers univariados detectados (IQR/Z-score)
- [ ] Outliers multivariados analizados (Isolation Forest)
- [ ] Clusters naturales descubiertos (K-means/DBSCAN)
- [ ] Correlaciones evaluadas (heatmap)
- [ ] Contexto de negocio consultado
- [ ] Decisiones sobre outliers documentadas
- [ ] Impacto en estadísticas validado

---

## 9. Resumen

**Patrones detectables:**
- ✅ **Temporales** → Tendencias, estacionalidad, ciclos
- ✅ **Clustering** → Agrupaciones naturales (segmentos)
- ✅ **Correlaciones** → Relaciones entre variables
- ✅ **Secuencias** → Eventos que ocurren juntos

**Métodos de detección de outliers:**
- ✅ **IQR** → Clásico, robusto, univariado
- ✅ **Z-Score** → Para distribuciones normales
- ✅ **Isolation Forest** → Multivariado, machine learning
- ✅ **DBSCAN** → Clustering + outliers simultáneo

**Framework de decisión:**
```
¿Es físicamente posible? → NO → ELIMINAR
            ↓ SÍ
¿Tiene sentido en negocio? → NO → INVESTIGAR
            ↓ SÍ
¿Es evento genuino raro? → SÍ → MANTENER/SEPARAR
            ↓ NO
            WINSORIZAR/TRANSFORMAR
```

> **Conclusión:** Los patrones te dicen qué esperar. Los outliers te dicen cuándo esperar lo inesperado. Dominar ambos transforma un analista junior en senior.

---

## 10. Referencias

### Vídeos
- [Outlier Detection Explained](https://youtu.be/example1) - Métodos visuales
- [Pattern Recognition in Data](https://youtu.be/example2) - Técnicas avanzadas
- [Netflix Data Analysis](https://youtu.be/example3) - Caso real

### Lecturas
- [Scikit-learn Outlier Detection](https://scikit-learn.org/stable/modules/outlier_detection.html)
- [Time Series Patterns](https://otexts.com/fpp2/patterns.html)
- [Anomaly Detection Survey](https://arxiv.org/abs/1901.03407)

### Herramientas
- [PyOD](https://github.com/yzhao062/pyod) - Librería especializada en outliers
- [Prophet](https://facebook.github.io/prophet/) - Patrones temporales (Facebook)
- [mlxtend](http://rasbt.github.io/mlxtend/) - Market Basket Analysis